In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
import pandas as pd
import numpy as np

In [ ]:
df = pd.read_csv("/kaggle/input/playground-series-s4e8/train.csv")
dt_test = pd.read_csv("/kaggle/input/playground-series-s4e8/test.csv")
ds_sub = pd.read_csv("/kaggle/input/playground-series-s4e8/sample_submission.csv")
df

In [ ]:
df.info(),df.value_counts()

In [ ]:
data = df.copy()
data_test = dt_test.copy()

In [ ]:
data = data.drop(columns=['id','stem-root','stem-surface','veil-type','veil-color','spore-print-color','gill-spacing','cap-surface','gill-attachment'])
data_test = data_test.drop(columns=['id','stem-root','stem-surface','veil-type','veil-color','spore-print-color','gill-spacing','cap-surface','gill-attachment'])
data.isnull().sum(),data.info()

In [ ]:
Obj_col = data.select_dtypes(include=['object']).columns
data['cap-diameter'] = data['cap-diameter'].fillna(6.309848e+00)

Obj_col_test = data_test.select_dtypes(include=['object']).columns
data_test['cap-diameter'] = data_test['cap-diameter'].fillna(6.309848e+00)

In [ ]:
from sklearn.preprocessing import OrdinalEncoder,LabelEncoder
le = LabelEncoder()
data['class'] = le.fit_transform(data['class'])
col_train =  Obj_col[Obj_col != 'class']
Od = OrdinalEncoder( handle_unknown='use_encoded_value',unknown_value=-1)
data[col_train] = Od.fit_transform(data[col_train].astype(str))
data

col_train_test =  Obj_col_test[Obj_col_test != 'class']
Od = OrdinalEncoder( handle_unknown='use_encoded_value',unknown_value=-1)
data_test[col_train_test] = Od.fit_transform(data_test[col_train_test].astype(str))

In [ ]:
import seaborn as sns 
import matplotlib.pyplot as plt 
plt.figure(figsize=(13, 13))

sns.set_style('dark')

sns.pairplot(data=df.iloc[:200], hue='class', palette='bright')

In [ ]:
#sample_data = data.sample(frac=0.5, random_state=42)
#sample_data_test = data_test.sample(frac = 0.5, random_state = 42)

In [ ]:
X = data.drop(columns=['class'])
y = data['class']
from sklearn.model_selection import train_test_split
X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.2, random_state=42)
X_train

In [ ]:
from tensorflow.keras.models import Sequential, load_model
from tensorflow.keras.layers import Dense, Dropout, Input, BatchNormalization
from tensorflow.keras.callbacks import EarlyStopping, LearningRateScheduler
import tensorflow as tf
from tensorflow.keras.optimizers import Adam

input_shape = X_train.shape[1:]
model = Sequential([
    Input(shape=input_shape)  
])


model.add(Dense(512, activation='relu'))
model.add(BatchNormalization())

model.add(Dropout(0.25))  

model.add(Dense(256, activation='relu'))
model.add(BatchNormalization())
model.add(Dropout(0.25))

model.add(Dense(128, activation='relu'))
model.add(BatchNormalization())
model.add(Dropout(0.25))


model.add(Dense(1, activation='sigmoid'))

optimizer = Adam(learning_rate=0.001)  
model.compile(loss='binary_crossentropy', optimizer=optimizer, metrics=['accuracy'])

early_stopping = EarlyStopping(monitor='val_loss', patience=10, restore_best_weights=True)
learning_rate_scheduler = LearningRateScheduler(lambda epoch: 0.001 * (0.8 ** (epoch // 10)))

history = model.fit(X_train, y_train,
                    epochs=50,
                    batch_size=128,
                    validation_data=(X_val, y_val), 
                    callbacks=[early_stopping, learning_rate_scheduler])


In [ ]:
y_test_pred = model.predict(data_test)
y_test_pred_class = np.where(y_test_pred == 1, 'e', 'p')
print(len(y_test_pred))
print(len(data_test))

In [ ]:
y_test_pred_class = y_test_pred_class.ravel()
submission = pd.DataFrame({'id': data_test.index, 'class': y_test_pred_class})
submission.head()

In [ ]:
submission.to_csv('submission.csv', index=False)